<a href="https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/Query_CTR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Búsquedas WOS+SCI+SCP+PTJ+CTR para UdeA

Búsquedas en bases bibligráficas  
* Web of Science (WOS), 
* Scielo (SCI)
* Scopus  (SCP)
* Puntaje (UDEA)
* Center (CTR)
de los artículos científicos de la UdeA

La base de datos se creó con:

[WOS_SCI_SCP_PTJ_CTR.ipynb](./WOS_SCI_SCP_PTJ_CTR.ipynb)

In [1]:
import os
VERSION='NEW'
if os.getcwd()=='/content':
    !pip install openpyxl xlrd wosplus fuzzywuzzy[speedup] > /dev/null

## functions

In [216]:
import pandas as pd
import wosplus as wp
pd.set_option('display.max_colwidth',200)
from venn import draw_venn, generate_colors
import numpy as np
import fuzzywuzzy.process as fwp
from fuzzywuzzy import fuzz
import re
import json
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import unidecode
import itertools
import sys
idc='CÉDULA'
# GENERAL PORPOSE FUNCTIONS
def split_names(s):
    """
    Extract the parts of the full name s.
    
    Works with:
    ----
        s='LA ROTTA FORERO DANIEL ANDRES'
        s='MONTES RAMIREZ MARIA DEL CONSUELO'
        s='CALLEJAS POSADA RICARDO DE LA MERCED'
        s='DE LA CUESTA BENJUMEA MARIA DEL CARMEN'
        s='JARAMILLO OCAMPO NICOLAS CARLOS MARTI'
        s='RESTREPO QUINTERO DIEGO ALEJANDRO'
        s='RESTREPO QUINTERO DIEGO'
        s='RESTREPO DIEGO'
    Fails with: 
    ----
        s='RANGEL MARTINEZ VILLAL ANDRES MAURICIO'
        s='RESTREPO DIEGO ALEJANDRO'
    """
    s=s.title()
    sl=re.sub('(\s\w{1,3})\s',r'\1-',s,re.UNICODE)
    sl=re.sub('^(\w{1,3})\s',r'\1-' ,sl,re.UNICODE)
    #if sl.find('-')>-1:
    sll=[s.replace('-',' ') for s in sl.split()]
    if s.split()==2:
        sll=[s.split()[0]]+['']+[s.split()[1]]
    #
    d={'NOMBRE COMPLETO' : ' '.join(sll[2:]+sll[:2]),
     'PRIMER APELLIDO' : sll[0], 
     'SEGUNDO APELLIDO': sll[1], 
     'NOMBRES'         :' '.join(sll[2:]), 
     'INICIALES'       :' '.join( [i[0]+'.' for i in ' '.join(sll[2:]).split() ] )
    }
    return d

##  Configure public links of  files in Google Drive
* If it is a Google Spreadsheet the corresponding file is downloaded as CSV
* If it is in excel/json or text file the file is downloaded  directly

To define your  own labeled IDs for public google drive files edit the next cell:

In [3]:
%%writefile drive.cfg
[FILES]
WOS_SCI_SCP_PTJ_CTR.json.gz=19E1C1kRk4I0V3uXojqko8-NEicWaPp1j
Base_de_datos_investigadores_Definitiva.csv=12oalgUeKhpvzkTPBP8pXCeHTrF-KO223dy9ov9w9QKs
produccion_fecha_vig_2003_2018.xlsx=1WbtX4K__TTLxXRjuLvqUYz9tuHCIlS5v
producción_reconocida_2002_2016_doi.xlsx=0BxoOXsn2EUNIY2lzVmNMT0VXaGs
oaudea.xlsx             = 1CcwobiEFACIbffNzNdLxpdxQukr8cZ5x

Overwriting drive.cfg


##  Load data bases

In [4]:
affil='Univ Antioquia'
drive_files=wp.wosplus('drive.cfg')

In [257]:
UDEAjsonfile='WOS_SCI_SCP_PTJ_CTR.json.gz'
tmp=drive_files.load_biblio(UDEAjsonfile,compression='gzip')
UDEA=drive_files.biblio['WOS'].copy().reset_index(drop=True)

In [15]:
AU=drive_files.read_drive_excel('Base_de_datos_investigadores_Definitiva.csv')

In [5]:
PTJ=drive_files.read_drive_excel('produccion_fecha_vig_2003_2018.xlsx')
AU_PTJ=PTJ[['cedula','nombre']].drop_duplicates()

In [29]:
LOCAL=False
if LOCAL:
    UDEA.to_json('WOS_SCI_SCP_PTJ_CTR.json.gz',orient='records')
    AU.to_json('AU.json')
    AU_PTJ.to_json('AU_PTJ.json',orient='records')

In [237]:
LOCAL_LOAD=True
if LOCAL_LOAD:
    UDEA=pd.read_json('WOS_SCI_SCP_PTJ_CTR.json.gz')
    AU=pd.read_json('AU.json').reset_index(drop=True)
    AU_PTJ=pd.read_json('AU_PTJ.json').reset_index(drop=True)

In [238]:
#UDEA=pd.read_json('/home/restrepo/Downloads/WOS_SCI_SCP_PTJ_CTR.json.gz').reset_index(drop=True)

In [239]:
UDEA[UDEA['UDEA_authors']!=''].reset_index(drop=True).shape

(12270, 181)

In [240]:
UDEA.shape

(15644, 181)

## Normalizations

### Relacionada con autores

Normalize `'AU'`

In [241]:
UDEA['AU']=UDEA['AU'].apply(lambda l: l if re.search('\n$',l) 
                 else 
                 '\n'.join(
            [
    re.sub( '([\w\-\s]+)(\s[\w]\.)',r'\1,\2',s ,re.UNICODE   ) 
                 for s in l.split(', ')]+['']
           ).replace('.','') )

In [242]:
UDEA[~UDEA['AU'].str.contains('\n$')].shape

(0, 181)

Normalize `'authors_WOS'`

In [243]:
#Specific entries
#('mcewen, juan g', 65) → mc ewen
#'hincape, me' → hincapie
def fix_WOS_author_Trujilloii(l):
    for i in range(len(l)):
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Trujilloii, S. B.':
            l[i]={'WOS_author': 'Trujillo, S. B.',
                  'affiliation': [
                    'Facultad de Medicina, Sede de Investigación Universitaria (SIU), Univ Antioquia, Medellín, Colombia'
                   ],
                  'i': 3}
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Manuel, J':
            l[i]={'WOS_author':'Senior Sanchez, Juan Manuel'}
            
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Balthazar, Vital':
            l[i]={'WOS_author': 'Baltazar, Vital',
                  'affiliation': ['Univ Antioquia, Colombia.'],
                  'i': 4}
        if l[i].get('WOS_author') and l[i].get('WOS_author')=='Oliviera-Angel, M':
            l[i]={'WOS_author': 'Olivera-Angel, M',
              'affiliation': ['Univ Antioquia, Fac Ciencias Agrarias Fisiol & Biotecnol Reprod, Antioquia, Colombia.']}            
    return l
UDEA['authors_WOS']=UDEA['authors_WOS'].apply( fix_WOS_author_Trujilloii )

In [244]:
def add_affilitions_to_authors_WOS(row_authors_WOS,row_C1,row_SCP_Affiliations):
    l=row_authors_WOS
    #Get rid of multiple author affiliations
    #TODO: Count multiple author affiliations
    row_C1=re.sub(r'\[[\w\.\s,]+\n[\w\.\s,]+\]','',row_C1,re.UNICODE)
    afwos=row_C1.strip().split('\n')
    afscp=row_SCP_Affiliations.strip().split('; ')
    if len(afwos)==1:
        af=re.sub('\[.*\]\s*','',afwos[0])
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                l[i].update({'affiliation':[af]})

    if len(afwos)==len(l):
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                af=re.sub('\[.*\]\s*','',afwos[i])
                l[i].update({'affiliation':[af]})
            
    elif len(afscp)==len(l):
        for i in range( len(l) ):
            if not l[i].get('affiliation'):
                l[i].update({'affiliation':[afscp[i]]})
    return l

In [245]:
def missing_authors_WOS(row):
    '''
    if row 'authors_WOS' is empty, try to fill it
    with AU and C1.
    if not empty just return it
    '''
    l=[]
    if len(row['authors_WOS'])==0:
        if row['AU']:
            l=[ {'WOS_author':a} for a in row['AU'].strip().split('\n') ]
        #Affiliations:
        row_C1=row['C1']
        row_SCP_Affiliations=row['SCP_Affiliations']
        l=add_affilitions_to_authors_WOS(l,row_C1,row_SCP_Affiliations)
    else:
        l=row['authors_WOS']
                     
    return l

In [246]:
UDEA['authors_WOS']=UDEA.apply(missing_authors_WOS,axis='columns')

In [247]:
def check_AU_authors(AU1,AU2,fuzz_partial_ratio=60):
    '''
    Check if two authors in the WOS AU format are the same
    '''
    AU1_eq_AU2=False
    #Check last name
    au1=AU1.split(', ')
    au2=AU2.split(', ')
    au1=[unidecode.unidecode(a.lower()) for a in au1]
    au2=[unidecode.unidecode(a.lower()) for a in au2]
    ln=np.intersect1d(au1[0].split(),au2[0].split()).shape[0]
    if ln>0:
        fn=np.intersect1d(  
            [ l[0] for l in au1[-1].split()],
            [l[0] for l in au2[-1].split()] ).shape[0]
        if fn>0:
            if fuzz.partial_ratio( AU1,AU2 )>fuzz_partial_ratio:
                AU1_eq_AU2=True
    return AU1_eq_AU2

def AU_to_authors_WOS(row,DEBUG=False):
    '''
     Compare 'WOS_author' → 'authors_WOS' with
             'AU' splitted list
     and fill for the missing 'authors_WOS' and
                              'affiliations'
    '''
    auwos=[d.get('WOS_author') for d in row['authors_WOS']]
    aurow=row['AU'].strip().split('\n')
    if len(auwos)<len(aurow):
        for au in aurow:
            NEW_AU=True
            for ws in auwos:
                if DEBUG:
                    print(au,'::',ws,'→',  check_AU_authors( au,ws) )
                if check_AU_authors( au,ws):
                    NEW_AU=False
                    break
            if NEW_AU:
                row['authors_WOS'].append(  
                    {'WOS_author':au})
        row['authors_WOS']=add_affilitions_to_authors_WOS(row['authors_WOS'],
                                       row['C1'],
                                       row['SCP_Affiliations'])
    return row['authors_WOS']

In [248]:
UDEA['authors_WOS']=UDEA.apply(AU_to_authors_WOS,axis='columns')

Normalize `'UDEA_authors'`

In [249]:
def fix_long_names(l):
    if len(l)>0:
        for i in range( len(l) ):
            if  l[i].get('full_name') and len(l[i].get('full_name').split())>4:
                dd=split_names(l[i].get('full_name'))
                l[i].update(dd)
    return l
#UDEA[ UDEA['UDEA_authors'].apply(lambda l: [d.get('CÉDULA')==22227944 for d in l] 
#   if l else [False]).apply(lambda l: True in l) ].loc[1573,'UDEA_authors']

In [257]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply( fix_long_names  )

In [258]:
def drop_duplicates(l):
    """
    Find duplicates in UDEA_authors by 'CÉDULA'
    """
    if len(l)>1:
        #Try to find duplicates j-times
        for j in range( int((len(l)+1)/2)   ):
            c=[]
            #In each try remove duplicated
            for i in range(len(l)):
                if l[i].get('CÉDULA') in c:
                    l.pop(i)
                    break
                else:
                    c.append(l[i].get('CÉDULA'))
            #Exit when not more duplicate found
            if not c:
                break
    return l

In [259]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(drop_duplicates)

In [260]:
def fix_NOMBRE_COMPLETOS(l):
    #for d in l:
    for i in range(len(l)):
        if not l[i].get('NOMBRE COMPLETO'):
            d['NOMBRE COMPLETO']=''
            if l[i].get('NOMBRES'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+l[i].get('NOMBRES')
            if l[i].get('PRIMER APELLIDO'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+' '+l[i].get('PRIMER APELLIDO')
            if l[i].get('SEGUNDO APELLIDO'):
                l[i]['NOMBRE COMPLETO']=l[i]['NOMBRE COMPLETO']+' '+l[i].get('SEGUNDO APELLIDO')            
    return l

In [261]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_NOMBRE_COMPLETOS)

In [262]:
dfid=UDEA[['UDEA_nombre','UDEA_cedula']]
dfid=dfid[dfid['UDEA_cedula'].apply(lambda n: n!=0 if isinstance(n,int) else False)].reset_index(drop=True)
dfid=dfid[dfid['UDEA_nombre']!=''].drop_duplicates().reset_index(drop=True)
dfid.shape

def get_cedula(l,fullnames_with_id=dfid):
    #for d in l:
    for i in range(len(l)):
        if not l[i].get('CÉDULA') and l[i].get('full_name'):
            try:
                l[i]['CÉDULA']=dfid[ ( dfid['UDEA_nombre']==l[i].get('full_name')) 
                            & ( dfid['UDEA_cedula']!=0 ) 
                            ].reset_index(
                          drop=True)['UDEA_cedula'].loc[0]
            except KeyError:
                pass
    return l

In [263]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_cedula)

In [264]:
def get_missing_cedula(l,AU=AU):
    '''
    AU: Data Frame with author info
    Added Facultad y afiliacion WOS
    '''
    for i in range(len(l)):
        if not l[i].get('CÉDULA'):
            au=AU[AU['NOMBRE COMPLETO']==l[i].get('NOMBRE COMPLETO')
                 ].reset_index(drop=True)
            if not au.empty:
                au=au.loc[0]
                l[i]['CÉDULA']=au['CÉDULA']
                if not l[i].get('DEPARTAMENTO'):
                    l[i]['DEPARTAMENTO']=au['DEPARTAMENTO']
                if not l[i].get('FACULTAD'):
                    l[i]['FACULTAD']=au['FACULTAD']
                if not l[i].get('NOMBRES'):
                    l[i]['NOMBRES']=au['NOMBRES']
                if not l[i].get('PRIMER APELLIDO'):
                    l[i]['PRIMER APELLIDO']=au['PRIMER APELLIDO']
                if not l[i].get('SEGUNDO APELLIDO'):
                    l[i]['SEGUNDO APELLIDO']=au['SEGUNDO APELLIDO']
                if not l[i].get('INICIALES'):
                    s=au['NOMBRES']
                    l[i]['INICIALES']=' '.join(
                        [ I[0]+'.' for I in s.split() ])
    return l

In [265]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(get_missing_cedula)

In [266]:
def fix_cedula(l,df=AU_PTJ,full_name='nombre',identification='cedula'):
    '''
    Get missing identification from an external database with
    the full name in the proper format:
     `full name`: LAST NAME FIRST NAME
    For each row, rebuild the list of dictionaries
    and if CÉDULA is missing try to fix it
    '''
    for i in range(len(l)):
        if not l[i].get('CÉDULA'):
            try:
                l[i]['CÉDULA']=df[df[nombre]==l[i].get('full_name')
                     ].reset_index(drop=True).loc[
                    0,identification]
            except:
                pass
    return l

In [267]:
UDEA['UDEA_authors']=UDEA['UDEA_authors'].apply(fix_cedula)

In [268]:
UDEA['C1']=UDEA['C1'].str.replace('deAntioquia','de Antioquia')

In [301]:
def add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,df,j):
    if not l[i].get('WOS_author'):
        l[i]['WOS_author']=[df.loc[j,'WOS_author']]
    if 'UDEA_affiliation' in df.columns:
        if not l[i].get('UDEA_affiliation'):
            l[i]['UDEA_affiliation']=df.loc[j,'affiliation']
    return l[i]

def compare_names(l,i,ll,j,smin=70,DEBUG=False):
    '''
    Compare dictionaries with UDEA_authors keys
    with DataFrame ll authors_WOS columns
    1. Select first matching first last name
       a. If failed select first mathcing second last name
          with extra quality check in full name
    2. First mathching names
       a. If failed first matching initials
    '''
    #j=-1 initialized in main function
    if DEBUG: print('i:',i)
    #Find First last name
    r=process.extractOne( unidecode.unidecode( l[i].get('PRIMER APELLIDO').lower() ),  
                       ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                    ).str.split(', ').str[0],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    if r[1]==100:
        j=r[2]
        
    #If failed, try with the second last name
    if r[1]<100:
        r=process.extractOne( unidecode.unidecode( l[i].get('SEGUNDO APELLIDO').lower() ),  
                           ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.split(', ').str[0],scorer=fuzz.partial_ratio )
        #Additional quality check
        j=r[2]
        s=fuzz.token_set_ratio( 
            unidecode.unidecode( l[i].get('NOMBRE COMPLETO').lower().replace('.','') ) ,
            unidecode.unidecode(ll['WOS_author'].loc[j].lower().replace('.','')  ) )
        if DEBUG: print(r)
        #print(s)
        if r[1]==100 and s>smin:
            j=r[2]
        else:
            j=-1
        
    #For names filter Initials:
    kk=ll[ll['WOS_author'].str.lower().apply(unidecode.unidecode
                        ).str.split(
                    ', ').str[-1].str.replace(
                 '\.','').apply(len)>2]
    if not kk.empty:
        if DEBUG: print(unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[0] ) )
        r=process.extractOne( unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[0] )
                           ,  kk['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if r[1]==100 and r[2]==j:
            if DEBUG: print('found FN j=',r[2])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
            return l,i,ll,j

        if DEBUG: print(r)
        #if not 100 try second name: fail proof
        if DEBUG: print(unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[-1]) ) 
        r=process.extractOne( unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[-1] )
                           ,  kk['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if DEBUG: print(r)
        if r[1]==100 and r[2]==j:
            if DEBUG: print('found SN j=',r[2])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)        
            return l,i,ll,j
    
    #if not 100 try first intial: fail proof
    if DEBUG: print(unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[0]) )
    r=process.extract( unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[0]) 
                       ,  ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                            ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found FI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)        
        return l,i,ll,j

    #if not 100 try second intial: fail proof
    if DEBUG: print(unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[-1]) )
    r=process.extract( unidecode.unidecode( l[i].get('INICIALES').lower().replace('.','').split(' ')[-1]) 
                       ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found SI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
        return l,i,ll,j

    #Some times the first last are confused with the first names
    #'Luis Fernando Restrepo Betancur' → 'Luis Restrepo, B.''
    # One of the last name is already covered so only the initials are affected
    #if not 100 try second intial: fail proof
    r=process.extract( unidecode.unidecode( l[i].get('PRIMER APELLIDO').lower()[0] )
                       ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
    if DEBUG: print(r)
    rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
    if rrr and rrr[0][0]==100:
        if DEBUG: print('found SI j=',rrr[0][1])
        l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
        return l,i,ll,j

    if l[i].get('SEGUNDO APELLIDO'):
        r=process.extract( unidecode.unidecode( l[i].get('SEGUNDO APELLIDO').lower()[0] ) 
                           ,ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                            ).str.replace('\.','').str.split(', ').str[-1],scorer=fuzz.partial_ratio )
        if DEBUG: print(r)
        rrr=[(rr[1],rr[2]) for rr in r if rr[2]==j]
        if rrr and rrr[0][0]==100:
            if DEBUG: print('found SI j=',rrr[0][1])
            l[i]=add_WOS_author_and_WOS_affiliation_to_UDEA_authores(l,i,ll,j)
            return l,i,ll,j
                      
    
    
    #print('final result: extract WOS_author and affiliauntion as lists:',lll[j])

    return l,i,ll,j

In [270]:
def missing_wos_authors_in_udea_authors(row):
    '''
    Compare UDEA_authors with authors_WOS and
    fill: 
    A) mising 'WOS_author' list in  UDEA_authors
    B) mising 'affiliation' list in  UDEA_authors
    1. Select first matching first last name
       a. If failed select first mathcing second last name
          with extra quality check in full name
    2. First mathching names
       a. If failed first matching initials
    '''    
    i=0
    j=-1
    l  =row['UDEA_authors']
    lll=row['authors_WOS']
    if lll:
        ll=pd.DataFrame(lll)
    else:
        ll=pd.DataFrame()
    #if not ll.empty:
    for i in range(len(l)):
        if not l[i].get('WOS_author'):
            l,i,ll,j=compare_names(l,i,ll,j,DEBUG=False)
    return l

In [271]:
UDEA['UDEA_authors']=UDEA.apply(missing_wos_authors_in_udea_authors,axis='columns')

Other normalizations

In [272]:
i=1
UDEA.loc[[i]].apply(missing_wos_authors_in_udea_authors,axis='columns').loc[i]

[{'CÉDULA': 32508663,
  'INICIALES': 'M. D. C.',
  'NOMBRE COMPLETO': 'Maria Del Consuelo Montes Ramirez',
  'NOMBRES': 'Maria Del Consuelo',
  'PRIMER APELLIDO': 'Montes',
  'SEGUNDO APELLIDO': 'Ramirez',
  'WOS_affiliation': ['SIU, Univ Antioquia, Environm Catalysis Res Grp, Medellin 1226, Colombia.',
   'Univ Antioquia, SIU, Environm Catalysis Res Grp, Medellin, Colombia.'],
  'WOS_author': ['Montes, Consuelo'],
  'full_name': 'MONTES RAMIREZ MARIA DEL CONSUELO'}]

In [273]:
lll=UDEA.loc[i,'authors_WOS']
ll=pd.DataFrame(lll)
l=UDEA.loc[i,'UDEA_authors']

In [274]:
l

[{'CÉDULA': 32508663,
  'INICIALES': 'M. D. C.',
  'NOMBRE COMPLETO': 'Maria Del Consuelo Montes Ramirez',
  'NOMBRES': 'Maria Del Consuelo',
  'PRIMER APELLIDO': 'Montes',
  'SEGUNDO APELLIDO': 'Ramirez',
  'WOS_affiliation': ['SIU, Univ Antioquia, Environm Catalysis Res Grp, Medellin 1226, Colombia.',
   'Univ Antioquia, SIU, Environm Catalysis Res Grp, Medellin, Colombia.'],
  'WOS_author': ['Montes, Consuelo'],
  'full_name': 'MONTES RAMIREZ MARIA DEL CONSUELO'}]

In [275]:
lll

[{'WOS_author': 'Hoyos Barreto, Andres Emilio',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.'],
  'i': 0},
 {'WOS_author': 'Jimenez Correa, Monica Maria',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.'],
  'i': 1},
 {'WOS_author': 'Munoz, AO',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.']},
 {'WOS_author': 'de Correa, CM',
  'affiliation': ['Univ Antioquia, Bogota, Colombia.']}]

In [276]:
ii=0
r=process.extractOne( unidecode.unidecode( 
            (l[ii].get('PRIMER APELLIDO')+l[ii].get('SEGUNDO APELLIDO')).lower() ),  
                           ll['WOS_author'].str.lower().apply(unidecode.unidecode
                                        ).str.replace('\-','').str.split(', ').str[0],scorer=fuzz.partial_ratio )
r

('jimenez correa', 48, 1)

Other normalizations

In [277]:
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCP_DOI'],axis=1 ).str.strip().str.lower()
UDEA['DI']=UDEA.apply( 
     lambda row: row['DI'] if row['DI'] else row['SCI_DI'],axis=1 ).str.strip().str.lower()

In [278]:
def normalize_NAME(row,WOS_NAME,SCI_NAME,SCP_NAME,UDEA_NAME,
                   SCI_sub=('',''),SCP_sub=('',''),UDEA_sub=('',''),
                  DEBUG=False):
    '''
    If the WOS of Science column is empty, copy the
    data from a filled column either from 
    * SCI
    * SCP
    * UDEA
    '''
    sn=row[WOS_NAME]
    if not sn:
        if SCI_NAME and row[SCI_NAME]:
            sn=re.sub( SCI_sub[0],SCI_sub[1], str( row[SCI_NAME] ) )
            if DEBUG:
                print('SCI: {}:{}'.format(row[SCI_NAME], sn))
        elif SCP_NAME and row[SCP_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[SCP_NAME] ) )
        elif UDEA_NAME and row[UDEA_NAME]:
            sn=re.sub( SCP_sub[0],SCP_sub[1], str( row[UDEA_NAME] ) )
            if DEBUG:
                print('UDEA: {}:{}'.format(row[UDEA_NAME], sn))            
    return sn

In [279]:
WOS_NAME ='SN'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_SN',
                                      SCP_NAME ='SCP_ISSN',
                                      UDEA_NAME='UDEA_issn rev',
                                      SCP_sub=( '^([\w]{4})([\w]{4})$',r'\1-\2\n'  ),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [280]:
WOS_NAME ='DI'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_DI',
                                      SCP_NAME ='SCP_DOI',
                                      UDEA_NAME='UDEA_doi')
                      ,axis='columns')

In [281]:
WOS_NAME ='VL'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_VL',
                                      SCP_NAME ='SCP_Volume',
                                      UDEA_NAME='',
                                      SCI_sub=('(\w)$',r'\1\n'),
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [282]:
WOS_NAME ='IS'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_IS',
                                      SCP_NAME ='SCP_Issue',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [283]:
WOS_NAME ='BP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_BP',
                                      SCP_NAME ='SCP_Page start',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [284]:
WOS_NAME ='EP'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_EP',
                                      SCP_NAME ='SCP_Page end',
                                      UDEA_NAME='',
                                      SCP_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

In [285]:
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'Spanish' if s=='ESPA?' else s)
UDEA['UDEA_idioma']=UDEA['UDEA_idioma'].apply(lambda s: 'English' if s=='INGLE' else s)
WOS_NAME ='LA'
UDEA[WOS_NAME]=UDEA.apply(lambda row: 
                       normalize_NAME(row,WOS_NAME,
                                      SCI_NAME ='SCI_LA',
                                      SCP_NAME ='SCP_Language of Original Document',
                                      UDEA_NAME='UDEA_idioma',
                                      SCI_sub =('\t',r''),
                                      SCP_sub =('(\w)$',r'\1\n'),
                                      UDEA_sub=('(\w)$',r'\1\n')
                                     )
                      ,axis='columns')

## TMP
Strategy:
* First assign a quality score, and lates try to discard bad quality rows

In [586]:
#Test: i=1, 378

In [751]:
def Quality_UDEA_authors(row,maxlmin=74,DEBUG=False):
    '''
    Check the Quality for the match 
      UDEA_authors 
    with 
      authors_WOS
    '''
    wos_list =[ unidecode.unidecode( d.get('WOS_author').lower(
                                     ).replace('.',''
                                     ).replace('-',' ')
                                   )
               for d in row.get('authors_WOS')]

    #If WOS_author not in row.get('UDEA_authors') is usually to be droppend
    
    
    #udea_list=[ [unidecode.unidecode( dd.lower().replace('.','') ) 
    #             for dd in d.get('WOS_author')] 
    #               for d in row.get('UDEA_authors')]
    udea_list=[ 
            [
              unidecode.unidecode( dd.lower(
                                     ).replace('.',''
                                     ).replace('-',' ')
                                 ) 
              for dd in d.get('WOS_author')
            ]
                if d.get('WOS_author') 
                else 
            [      
                unidecode.unidecode(
                  '{} {}, {}'.format(d.get('PRIMER APELLIDO'),
                                     d.get('SEGUNDO APELLIDO'),
                                     d.get('NOMBRES')
                                    ).replace(' ,',','
                                    ).strip(
                                    ).lower()
                  )
             ]
                  for d in row.get('UDEA_authors')
           ]    
    udea_list=[ list(set(l)) for l in udea_list  ]
    
    if not udea_list:
        if DEBUG: print('==1==')
        Q=True
        return Q
    for i in range(len(udea_list)): #=len(row['UDEA_authors'])
    #for l in udea_list: #=len(row['UDEA_authors'])
        Q=False
        for au in wos_list:
            #Exact match
            r=np.intersect1d( [au],udea_list[i] ).shape[0]
            if r>0:
                #l found!
                if DEBUG: print('==2==')
                Q=True
                break
            else:
                #Similarity match
                #print(au,udea_list[i])
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.partial_ratio)
                if rr[1]==100:
                    if DEBUG: print('==3==')
                    Q=True
                    break
                #print(rr)
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    if DEBUG: print('==4==')
                    Q=True
                    break
                #keep only the first letter of names
                fnau=re.sub('([\w\s]+\,\s\w)[\w\s]+',r'\1',au)
                l=[re.sub('([\w\s]+\,\s\w)[\w\s]+',r'\1',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(fnau,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    if DEBUG: print('==5==')
                    Q=True
                    break
                    
                #keep only first letter of second last name
                l=[re.sub('(\s\w)\w+\,',r'\1,',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(au,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    if DEBUG: print('==6==')
                    Q=True
                    break

                #remove first last name from udea_list[i] and the first letter of names
                l=[re.sub('^\w+\s(\w\w+\,\s\w)\w+',r'\1',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(au,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    if DEBUG: print('==7==')
                    Q=True
                    break
                #Very similar names in any order
                #print('*',au,udea_list[i])
                rr=process.extractOne(au,udea_list[i],scorer=fuzz.token_sort_ratio)
                if rr[1]>95:
                    if DEBUG: print('==8==')
                    Q=True
                    break
                #Keep only first last name and the first letter of names
                fnau= re.sub('(^\w\w+)\s\w+(\,\s\w)[\s\w]*',r'\1\2',au)
                l   =[re.sub('(^\w\w+)\s\w+(\,\s\w)[\s\w]*',r'\1\2',ll) for ll in udea_list[i]]
                #print(l)
                rr=process.extractOne(fnau,l,scorer=fuzz.token_set_ratio)
                if rr[1]==100:
                    if DEBUG: print('==9==')
                    Q=True
                    break
                
                #Q=False
        if not Q:
            maxl=0
            for j in range(len(udea_list[i])):
                rr=process.extractOne(udea_list[i][j],wos_list,scorer=fuzz.ratio)
                print(rr)
                if rr[1]>maxl:
                    maxl=rr[1]
            print(udea_list[i],i,j,maxl)
            #keep if maxl>maxlmin See step: ==10===
            if maxl>maxlmin:
                Q=True
                return Q
            #else remove
        
            
    return Q

In [752]:
i=378 #1,378
UDEA.loc[[i]].apply(lambda row:Quality_UDEA_authors(row,DEBUG=True),axis='columns')

('londono, charle', 41)
['lopera castano, mauricio'] 0 0 41


378    False
dtype: bool

In [724]:
ll='padilla ll, sonia c'
re.sub('(\s\w)\w+\,',r'\1,',ll)

'padilla l, sonia c'

In [725]:
row={}
row['authors_WOS' ]=UDEA.loc[i,'authors_WOS']
row['UDEA_authors']=UDEA.loc[i,'UDEA_authors']
wos_list =[ unidecode.unidecode( d.get('WOS_author'
                                     ).lower(
                                     ).replace('.',''
                                     ).replace('-',' '))
               for d in row.get('authors_WOS')]
udea_list=[ 
            [
              unidecode.unidecode( dd.lower(
                                     ).replace('.',''
                                     ).replace('-',' ') ) 
              for dd in d.get('WOS_author')
            ]
                if d.get('WOS_author') 
                else 
            [      
                unidecode.unidecode(
                  '{} {}, {}'.format(d.get('PRIMER APELLIDO'),
                                     d.get('SEGUNDO APELLIDO'),
                                     d.get('NOMBRES')
                                    ).replace(' ,',','
                                    ).strip(
                                    ).lower()
                  )
             ]
                  for d in row.get('UDEA_authors')
           ]

In [726]:
wos_list

['villa v, liliana', 'martinez h, eliana', 'franco m, diego f']

In [727]:
udea_list

[['martinez herrera, eliana'], ['villa v, liliana']]

In [668]:
UDEA.loc[i,'SCP_Authors'],UDEA.loc[i,'SCI_AU']

('', '')

In [670]:
ii=4
jj=2
au=wos_list[ii]
l=udea_list[jj]
print(au,l)
process.extractOne(au,l,scorer=fuzz.token_sort_ratio)

ros, john fredy nieto ['fredy nieto rios, john', 'nieto rios, j f', 'nieto rios, john fredy']


('fredy nieto rios, john', 98)

In [446]:
process.extractOne(au,l,scorer=fuzz.partial_ratio)

('villada zapata, johny', 82)

In [493]:
fuzz.partial_ratio('zapata j','zapata j v')

100

In [470]:
row['UDEA_authors'][0].get('SEGUNDO APELLIDO')+', '+row['UDEA_authors'][0].get('INICIALES')

'Zapata, J. A.'

In [445]:
process.extractOne(au,l,scorer=fuzz.partial_token_sort_ratio)

('villada zapata, johny', 70)

In [447]:
process.extractOne(au,l,scorer=fuzz.token_set_ratio)

('villada zapata, johny', 75)

In [441]:
process.extractOne('zapata, a r',l,scorer=fuzz.partial_token_set_ratio)

('villada zapata, johny', 100)

In [428]:
process.extractOne('robledo h, i j',l,scorer=fuzz.token_set_ratio)

('robledo, s', 88)

In [781]:
i=378
i=1909
i=1948
i=2133
i=2210
i=2376
i=2427
i=2484
i=2648
i=2763
i=2821
i=2872
i=2968
i=2974
UDEA=UDEA[UDEA['TI']!=UDEA.loc[i,'TI']].reset_index(drop=True)

In [287]:
def Quality_UDEA_authors_fail_safe(row):
    try:
        Q=Quality_UDEA_authors(row)
    except (NameError,TypeError):
        Q=False
    return Q

In [738]:
ll='ruiz cordoba, j'


'ruiz, j'

In [753]:
UDEA[5000:].apply(Quality_UDEA_authors,axis='columns')

('franco henao, leidy tatiana', 33)
('franco henao, leidy tatiana', 30)
('franco henao, leidy tatiana', 35)
['rodriguez gazquez, maria de los angeles', 'de los angeles rodriguez gazquez, maria', 'rodriguez gazquez, maria'] 0 2 35
('donado, juan r', 39)
['duque serna, francisco levi'] 0 0 39
('gomez, juan f', 47)
['alvarez gomez, gloria jeanethe'] 1 0 47
('arango barrientos, marcos', 38)
['coll jimenez, yadira esther'] 0 0 38
('ortiz barrientos, d', 41)
['soto calderon, ivan dario'] 0 0 41
('jaime, c f', 34)
['carmona fonseca, jaime de jesus'] 0 0 34
('agudeloa, a', 50)
('agudeloa, a', 77)
('agudeloa, a', 95)
('agudeloa, a', 71)
('agudeloa, a', 87)
['agudelo santamaria, andres felipe', 'agudelo, andres', 'agudelo, a', 'agudelo, andres f', 'agudelo, a f'] 0 4 95
('ramirez, robinson', 56)
('ramirez, robinson', 59)
('ramirez, robinson', 56)
['ramirez pineda, jose r', 'ramirez pineda, jose', 'ramirez pineda, j r'] 0 2 59
('arango barrientos, marcos', 30)
['velasquez trujillo, luz elena'] 0 

5000     True
5001     True
5002     True
5003     True
5004     True
5005     True
5006     True
5007     True
5008     True
5009     True
5010     True
5011     True
5012     True
5013     True
5014     True
5015     True
5016     True
5017     True
5018     True
5019     True
5020     True
5021     True
5022     True
5023     True
5024     True
5025     True
5026     True
5027     True
5028     True
5029     True
         ... 
15614    True
15615    True
15616    True
15617    True
15618    True
15619    True
15620    True
15621    True
15622    True
15623    True
15624    True
15625    True
15626    True
15627    True
15628    True
15629    True
15630    True
15631    True
15632    True
15633    True
15634    True
15635    True
15636    True
15637    True
15638    True
15639    True
15640    True
15641    True
15642    True
15643    True
Length: 10644, dtype: bool

In [295]:
#UDEA.loc[[5]].apply(Quality_UDEA_authors,axis='columns')
#UDEA.apply(Quality_UDEA_authors,axis='columns')
#UDEA[~UDEA.apply(Quality_UDEA_authors_fail_safe,axis='columns')].shape

NameError: ("name 'rr' is not defined", 'occurred at index 1')

In [784]:
row['authors_WOS' ]

[{'WOS_author': 'Rueda Nobmann, María Teresa',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Carvajal-Castrillón, Julián',
  'affiliation': ['Univ Antioquia, Colombia.']},
 {'WOS_author': 'Restrepo Pelaez, Adelaida',
  'affiliation': ['Universidad CES, Colombia.']},
 {'WOS_author': 'Dávila Plata, Diana',
  'affiliation': ['Universidad San Buenaventura, Colombia.']},
 {'WOS_author': 'Garzón Girlado, Luz Dey',
  'affiliation': ['Universidad San Buenaventura, Colombia.']},
 {'WOS_author': 'Galeano Toro, Luz Marina',
  'affiliation': ['Unidad de Neuropsicología del INDEC, Colombia.']},
 {'WOS_author': 'Arboleda Ramírez, Alejandra',
  'affiliation': ['Universidad CES, Colombia.']},
 {'WOS_author': 'Bareño Silva, José',
  'affiliation': ['Universidad San Buenaventura, Colombia.']}]

In [785]:
row['UDEA_authors']

[{'CÉDULA': 70878402.0,
  'DEPARTAMENTO': 'Instituto de Investigaciones Médicas',
  'FACULTAD': 'Facultad de Medicina',
  'GRUPO': 'Inmunodeficiencias Primarias',
  'INICIALES': 'D. C.',
  'NOMBRE COMPLETO': 'Daniel Camilo Aguirre Acevedo',
  'NOMBRES': 'Daniel Camilo',
  'PRIMER APELLIDO': 'Aguirre',
  'SEGUNDO APELLIDO': 'Acevedo',
  'full_name': 'AGUIRRE ACEVEDO DANIEL CAMILO'},
 {'CÉDULA': 7463551,
  'INICIALES': 'D. A.',
  'NOMBRE COMPLETO': 'David Antonio Pineda Salazar',
  'NOMBRES': 'David Antonio',
  'PRIMER APELLIDO': 'Pineda',
  'SEGUNDO APELLIDO': 'Salazar',
  'full_name': 'PINEDA SALAZAR DAVID ANTONIO'}]

In [751]:
AU[AU['NOMBRE COMPLETO'].str.lower().str.contains('usuga')]

,CÉDULA,DEPARTAMENTO,FACULTAD,GRUPO,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO
464,43276372,Departamento de Ingeniería Industrial,Facultad de Ingeniería,INCAS-Innovación y Gestión de Cadenas de Abastecimiento,Olga Cecilia Usuga Manco,Olga Cecilia,Usuga,Manco


In [758]:
AU_PTJ[AU_PTJ['nombre'].str.lower().str.contains('usuga')]

,cedula,nombre
392,43276372,USUGA MANCO OLGA CECILIA
487,43568255,JIMENEZ USUGA NORA DEL SOCORRO


In [671]:
AU_PTJ[AU_PTJ['cedula']==22227944]

,cedula,nombre
2199,22227944,MOLINA MARIN GLORIA DEL SOCORRO


In [527]:
UDEA[UDEA['UDEA_authors'].apply(
    lambda l: [d.get('CEDULA')==94226644 if d.get('CEDULA') else False 
               for d in l ] if l else [False]).apply(
    lambda l: True in l
                                                    )]

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_tipo mat,UDEA_tipo mov,UDEA_título,UDEA_título_list,UDEA_valor item,UT,VL,WC,Z9,authors_WOS


In [395]:
udea_list

[['carmona-fonseca, jaime',
  'carmona-fonseca, j.',
  'carmona, j.',
  'carmona fonseca, jaime',
  'carmona fonseca, jaime',
  'carmona-fonseca, jaime',
  'carmona- fonseca, jaime',
  'carmona, jaime'],
 ['uscategui, rosa m.',
  'uscategui, rosa magdalena',
  'uscategui, r. m.',
  'magdalena uscategui, rosa']]

In [413]:
if not udea_list: Q=True
for l in udea_list:
    Q=False
    for au in wos_list:
        #Exact match
        r=np.intersect1d( [au],l ).shape[0]
        if r>0:
            #l found!
            Q=True
            break
        else:
            #Similarity match
            print(rr)
            if rr[1]==100:
                Q=True
                break
            print(rr)
            if rr[1]==100:
                Q=True
                break
                
            Q=False
if Q:
    #return Q
    pass
Q

('uscategui, rosa magdalena', 100)
('uscategui, rosa magdalena', 100)


True

In [135]:
if not Q:
    for l in udea_list:
        Q=False
        for au in wos_list:
            if r[1]==100:
                Q=True
if Q:
    #return Q
    pass
Q                

In [87]:
np.intersect1d(['A'],['A']).shape[0]

1

In [53]:
i=0
UDEA.loc[[i]].apply(check_UDEA_authors)

,AB,AF,AR,AU,BA,BE,BF,BN,BP,C1,...,UDEA_tipo mat,UDEA_tipo mov,UDEA_título,UDEA_título_list,UDEA_valor item,UT,VL,WC,Z9,authors_WOS
0,"A new methodology for the evaluation of voltage sags and a decrease of the insufficiency found in some classification methodologies on literature are presented. To achieve this, the phasorial meth...","Garcia Quintero, Edwin\nFuster Roig, Vicente\n",,"Quintero, EG\nRoig, VF\n",,,,,196\n,"[Garcia Quintero, Edwin] Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.\nUniv Politecn Valencia, Dept Ingn Elect, E-46071 Valencia, Spain.\n",...,ARTREVA1,NO,"""METODOLOGIA PARA LA EVALUACION DE HUECOS DE TENSION"" (METHODOLOGY FOR EVALUATION OF VOLTAGE SAGS)","[METODOLOGIA PARA LA EVALUACION DE HUECOS DE TENSION, METHODOLOGY FOR EVALUATION OF VOLTAGE SAGS]",,WOS:000278425300019\n,,"Engineering, Multidisciplinary\n",0,"[{'affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.'], 'WOS_author': 'Garcia Quintero, Edwin', 'i': 0}, {'affiliation': ['Univ Politecn Valencia, Dept Ingn Elect, E-46071 Valen..."


## Centros

In [151]:
udea=UDEA.copy()

In [152]:
def convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(df,
                            json_column='UDEA_authors',dictionary_key='FACULTAD',sep='; '):
    return df[json_column].apply(lambda l:  
       [d.get(dictionary_key) for d in  l if type(l)==list and d.get(dictionary_key) ]  
               ).apply(
        pd.np.unique
        ).apply(sep.join)    

In [153]:
JSON=True
if not JSON:
    udea['FACULTAD']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='FACULTAD')
    udea['DEPARTAMENTO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='DEPARTAMENTO')
    udea['GRUPO']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='GRUPO')
    udea['autores_UDEA']=convert_dictionary_keys_of_list_of_dictionaries_to_column_dataframe(udea,
                                            json_column='UDEA_authors',dictionary_key='full_name')

# Prepare columns

In [154]:
def new_Autores(l,nombre='Nombres',tipo='Tipo',identificación='Identificación'):
    dd={}
    ll=[]
    for d in l:
        if d.get('NOMBRE COMPLETO'):
            dd[nombre]=d['NOMBRE COMPLETO']
            dd[tipo]='Profesor vinculado'
        if d.get('CÉDULA'):
            dd[identificación]=d['CÉDULA']
        ll.append(dd)
    return ll

In [155]:
if JSON:
    año='annio'; título='tituloArticulo'; revista='nombreRevista'
    issn='issn'; doi='doi'; volumen='volumen'; número='numero'
    páginas='paginas'; semestre='semestre'; idioma='idioma'
    tipoRevista='tipoRevista';     Colciencias='clasificacionColciencias'
    proyecto='proyectoAsociado'
    autores='autores'
else:
    año='Año';   título='Título del Artículo'; revista='Nombre de la Revista'
    issn='ISSN'; doi='DOI'; volumen='Volumen'; número='numero'
    páginas='Páginas'; semestre='Semestre'; idioma='Idioma'
    tipoRevista='Tipo de Revista'; Colciencias='Clasificación Colciencias'
    proyecto='Producto asociado a un proyecto de extensión o investigación'
    udea['Nombre del proyecto de extensión o investigación']=''
    autores='Autores'
    
udea[título]=udea['TI']
udea[revista]=udea['SO']
udea[issn]=udea['SN'].str.replace('\n','')
udea[doi]=udea['DI']
udea[volumen]=udea['VL'].str.replace('\n','')
udea[número]=udea['IS'].str.replace('\n','')
udea[páginas]=udea.apply(lambda row:
                  row['BP'].replace('\n','')+'-'+row['EP'].replace('\n','') 
                  if row['BP'] 
                  else row['BP'],axis='columns').str.replace('\-$','')
udea[año]=udea['PY']
udea[semestre]=None
udea[idioma]=udea['LA'].str.replace('\n','')
udea[tipoRevista]=udea.apply(lambda row: 'Revista Internacional Indexada' 
           if row.get('Tipo').find('WOS')>-1 or row.get('Tipo').find('SCP')
           else 'Revista Indexada en SCIELO',axis='columns')
udea[Colciencias]=None
udea[proyecto]=None


In [156]:
udea[autores]=udea['UDEA_authors'].apply(new_Autores)

In [157]:
udea_au=udea[udea[autores].apply(len)>0].reset_index(drop=True)

In [158]:
udea.PY.max()

2018

In [159]:
f=open('example.json','w')
f.write(
'''
[{
  "autores" : [ 
      {
          "nombreAutor" : "Jonathan Aguilar Bedoya",
          "identificacionAutor" : null,
          "tipoAutor" : "Estudiante de posgrado",
          "programaAcademico" : [ 
              "60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Ainhoa Rubio Celemente",
          "tipoAutor" : "Autor externo",
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ],
          "institucion" : "Tecnológico de Antioquia. Institución Universitaria",
          "pais" : "Colombia"
      }, 
      {
          "nombreAutor" : "Laura Isable Velasquez Garcia",
          "tipoAutor" : "Profesor ocasional",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }, 
      {
          "nombreAutor" : "Edwin Lenin Chica Arrieta",
          "identificacionAutor" : "78761560",
          "tipoAutor" : "Profesor vinculado",
          "programaAcademico" : [ 
              "516-INGENIERÍA MECÁNICA"
          ],
          "grupoInvestigacion" : [ 
              "Grupo de Energía Alternativa"
          ]
      }
  ],
  "tituloArticulo" : "Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine",
  "nombreRevista" : "Energies",
  "issn" : "1996-1073",
  "doi" : "10.3390/en12244679",
  "volumen" : 12,
  "numero" : 24,
  "paginas" : "1-18",
  "annio" : 2019,
  "semestre" : "2019-2",
  "idioma" : "Inglés",
  "tipoRevista" : "Revista Internacional Indexada",
  "clasificacionColciencias" : "A1",
  "proyectoAsociado" : null
}
]
'''
)
f.close()

In [160]:
if JSON:
    new_format=pd.read_json( 'example.json' )
else:
    new_format=pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vTiaxuZSGmI-aFgMUVjRAU3ws7WN9xmtjMEWu_SLOd5kAq_ZAuUtJUVr8qxNl3sMcp_fjE2gLVt_tdp/pub?output=xlsx')

In [161]:
audea=udea_au[list(new_format.columns)]
#.to_excel('UDEA_2018.xlsx',index=False)

In [162]:
audea.shape

(12270, 14)

# NOTA
Se ha añadido 
* `'Facultad'`
* y `'afiliacionWOS'` cuando no se conoce el departamento

In [372]:
kk=pd.DataFrame()
kk['NOMBRE COMPLETO']=UDEA['UDEA_authors'].apply(lambda l: [d.get('NOMBRE COMPLETO') for d in l if not d.get('CÉDULA')]
                          ).apply(lambda l: l[0] if l else None).dropna().reset_index(drop=True)

In [373]:
kk.merge(AU,on='NOMBRE COMPLETO',how='left').dropna()

,NOMBRE COMPLETO,CÉDULA,DEPARTAMENTO,FACULTAD,GRUPO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO


In [330]:
AU[AU['NOMBRE COMPLETO'].str.contains('Ocho')].reset_index(drop=True).loc[0,'CÉDULA']

71260216

In [356]:
d={'NOMBRE COMPLETO':'Elena Valentina Gutierrez Gutierrez','CÉDULA':29126775}
au=AU[AU['NOMBRE COMPLETO']==d.get('NOMBRE COMPLETO')].reset_index(drop=True).loc[0]
au.empty

False

In [362]:
s='Elena Valentina'


'E. V.'

In [163]:
def new_Autores(l,AU=AU,
                nombre='nombreAutor',#str 
                tipo='tipoAutor',
                identificación='identificacionAutor',
                facultad='facultad',#str
                departamento='departamento',#list
                grupo='grupoInvestigacion',#list
                institución='institucion'#str
               ):
    '''
    AU: Data Frame with author info
    Added Facultad y afiliacion WOS
    '''
    dd={}
    ll=[]
    for d in l:
        if d.get('CÉDULA'):
            au=AU[AU['CÉDULA']==int(d.get('CÉDULA'))].reset_index(drop=True)
            if not au.empty:
                dd[nombre]=au.loc[0,'NOMBRE COMPLETO']
                dd[tipo]='Profesor vinculado'
                dd[identificación]=au.loc[0,'CÉDULA']
                dd[facultad]=au.loc[0,'FACULTAD']
                dd[departamento]=[au.loc[0,'DEPARTAMENTO']]
                dd[grupo]=[au.loc[0,'GRUPO']]
                dd[institución]='Universidad de Antioquia'
                try:
                    dd['afiliacionWOS']=d.get('WOS_affiliation')[0]
                except:
                    pass
            else:
                if d.get('NOMBRE COMPLETO'):
                    dd[nombre]=d['NOMBRE COMPLETO']
                    dd[tipo]='Profesor vinculado'
                if d.get('CÉDULA'):
                    dd[identificación]=d['CÉDULA']
                    dd[institución]='Universidad de Antioquia'
                    try:
                        dd['afiliacionWOS']=d.get('WOS_affiliation')[0]
                    except:
                        pass
                else:
                    dd[identificación]=None
        ll.append(dd)
        dd={}
    return ll

In [164]:
audea[autores]=udea['UDEA_authors'].apply(new_Autores)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [39]:
udea[autores].loc[1]

[{'Identificación': 32508663,
  'Nombres': 'Del Consuelo Montes Ramirez',
  'Tipo': 'Profesor vinculado'}]

TODO:
* Adiciones `WOS_athors` to `UDEA_authors`
  1. partial ratio on last names -> 100
  1. partial ratio on initials  

In [781]:
[{'WOS_author': 'Grisales F, Lina María',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Giraldo G, Libardo Antonio',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 1}]

[{'WOS_author': 'Grisales F, Lina María',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Giraldo G, Libardo Antonio',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 1}]

In [783]:
[{'INICIALES': 'L. A.',
  'NOMBRE COMPLETO': 'Libardo Antonio Giraldo Gaviria',
  'NOMBRES': 'Libardo Antonio',
  'PRIMER APELLIDO': 'Giraldo',
  'SEGUNDO APELLIDO': 'Gaviria',
  'full_name': 'GIRALDO GAVIRIA LIBARDO ANTONIO'}]

[{'INICIALES': 'L. A.',
  'NOMBRE COMPLETO': 'Libardo Antonio Giraldo Gaviria',
  'NOMBRES': 'Libardo Antonio',
  'PRIMER APELLIDO': 'Giraldo',
  'SEGUNDO APELLIDO': 'Gaviria',
  'full_name': 'GIRALDO GAVIRIA LIBARDO ANTONIO'}]

In [784]:
[{'CÉDULA': 18927154.0,
  'DEPARTAMENTO': 'Departamento de Ingeniería Eléctrica',
  'FACULTAD': 'Facultad de Ingeniería',
  'GRUPO': 'Grupo de Investigación en Materiales y Sistemas Energéticos TESLA',
  'INICIALES': 'E.',
  'NOMBRE COMPLETO': 'Edwin Garcia Quintero',
  'NOMBRES': 'Edwin',
  'PRIMER APELLIDO': 'Garcia',
  'SEGUNDO APELLIDO': 'Quintero',
  'WOS_affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.',
   'Univ Antioquia, Colombia.',
   'Department of Electrical Engineering, Univ Antioquia, Colombia'],
  'WOS_author': ['Garcia Quintero, Edwin',
   'Garcia-Quintero, E.',
   'Garcia-Quintero, Edwin',
   'Garcia, Edwin'],
  'full_name': 'GARCIA QUINTERO EDWIN'}]

[{'CÉDULA': 18927154.0,
  'DEPARTAMENTO': 'Departamento de Ingeniería Eléctrica',
  'FACULTAD': 'Facultad de Ingeniería',
  'GRUPO': 'Grupo de Investigación en Materiales y Sistemas Energéticos TESLA',
  'INICIALES': 'E.',
  'NOMBRE COMPLETO': 'Edwin Garcia Quintero',
  'NOMBRES': 'Edwin',
  'PRIMER APELLIDO': 'Garcia',
  'SEGUNDO APELLIDO': 'Quintero',
  'WOS_affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.',
   'Univ Antioquia, Colombia.',
   'Department of Electrical Engineering, Univ Antioquia, Colombia'],
  'WOS_author': ['Garcia Quintero, Edwin',
   'Garcia-Quintero, E.',
   'Garcia-Quintero, Edwin',
   'Garcia, Edwin'],
  'full_name': 'GARCIA QUINTERO EDWIN'}]

In [236]:
    wos_list =[ unidecode.unidecode( 
                  d.get('WOS_author').lower())  
                  for d in row.get('authors_WOS')]
    wos_aff  =[ d.get('affiliation') for d in row.get('authors_WOS')]    
    l=row.get('UDEA_authors')
    ll=row.get('authors_WOS')
    if ll:
        ll=pd.DataFrame(ll)
    else:
        ll=pd.DataFrame()
    for i in range(len(l)):
        if not l[i].get('WOS_author'):
            

    udea_ln =[ unidecode.unidecode( 
                 d.get('PRIMER APELLIDO').lower())  for d in row.get('UDEA_authors')]
    udea_n  =[ unidecode.unidecode( 
                 d.get('NOMBRES').lower())  for d in row.get('UDEA_authors')]
    udea_in =[ unidecode.unidecode( 
                 d.get('INICIALES').lower())  for d in row.get('UDEA_authors')]
    print(wos_list)
    print(udea_ln)

IndentationError: expected an indented block (<ipython-input-236-656bed09c844>, line 16)

In [851]:
UDEA['authors_WOS'].loc[0]

[{'WOS_author': 'Garcia Quintero, Edwin',
  'affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.'],
  'i': 0},
 {'WOS_author': 'Roig, VF',
  'affiliation': ['Univ Politecn Valencia, Dept Ingn Elect, E-46071 Valencia, Spain.']}]

In [852]:
UDEA['UDEA_authors'].loc[0]

[{'CÉDULA': 18927154.0,
  'DEPARTAMENTO': 'Departamento de Ingeniería Eléctrica',
  'FACULTAD': 'Facultad de Ingeniería',
  'GRUPO': 'Grupo de Investigación en Materiales y Sistemas Energéticos TESLA',
  'INICIALES': 'E.',
  'NOMBRE COMPLETO': 'Edwin Garcia Quintero',
  'NOMBRES': 'Edwin',
  'PRIMER APELLIDO': 'Garcia',
  'SEGUNDO APELLIDO': 'Quintero',
  'WOS_affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.',
   'Univ Antioquia, Colombia.',
   'Department of Electrical Engineering, Univ Antioquia, Colombia'],
  'WOS_author': ['Garcia Quintero, Edwin',
   'Garcia-Quintero, E.',
   'Garcia-Quintero, Edwin',
   'Garcia, Edwin'],
  'full_name': 'GARCIA QUINTERO EDWIN'}]

In [946]:
l=[{'INICIALES': 'L. A.',
  'NOMBRE COMPLETO': 'Libardo Antonio Giraldo Gaviria',
  'NOMBRES': 'Libardo Antonio',
  'PRIMER APELLIDO': 'Giraldo',
  'SEGUNDO APELLIDO': 'Gaviria',
  'full_name': 'GIRALDO GAVIRIA LIBARDO ANTONIO'},
 {'CÉDULA': 43746089,
  'INICIALES': 'L. M.',
  'NOMBRE COMPLETO': 'Lina Maria Grisales Franco',
  'NOMBRES': 'Lina Maria',
  'PRIMER APELLIDO': 'Grisales',
  'SEGUNDO APELLIDO': 'Franco',
  'full_name': 'GRISALES FRANCO LINA MARIA'}]


In [947]:
lll=[{'WOS_author': 'Grisales F, María',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Giraldo G, Libardo Antonio',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 1}]

In [45]:
i=5742
l=UDEA['UDEA_authors'].loc[i]
l

[{'INICIALES': 'M.',
  'NOMBRE COMPLETO': 'Michael Hess',
  'NOMBRES': 'Michael',
  'PRIMER APELLIDO': 'Hess',
  'SEGUNDO APELLIDO': '',
  'full_name': 'HESS MICHAEL'}]

In [46]:
[ d.get('NOMBRE COMPLETO') for d in l]

['Michael Hess']

In [47]:
lll=UDEA['authors_WOS'].loc[i]
ll=pd.DataFrame(lll)

In [48]:
ll

,WOS_author,affiliation
0,"Mejia, AI","[Univ Antioquia, Fac Ciencias Exactas & Nat, Dept Quim, Lab Invest Polimeros, AA-1226 Medellin, Colombia.]"
1,"Lopez, BL","[Univ Antioquia, Fac Quim Farmaceut, Dept Farm, AA-1226 Medellin, Colombia.]"
2,"Mulet, A","[Univ Politecn Valencia, Dept Tecnol Alimentos, E-46071 Valencia, Spain.]"


In [1102]:
df=ll
'WOS_author' in df.columns

True

In [1103]:
for i in range(1):
     l,i,ll,j=compare_names(l,i,ll,j,DEBUG=True)   

i: 0
('mejia', 25, 0)
m
[('ai', 0, 0), ('bl', 0, 1), ('a', 0, 2)]
m
[('ai', 0, 0), ('bl', 0, 1), ('a', 0, 2)]
[('ai', 0, 0), ('bl', 0, 1), ('a', 0, 2)]


In [1104]:
l

[{'INICIALES': 'M.',
  'NOMBRE COMPLETO': 'Michael Hess',
  'NOMBRES': 'Michael',
  'PRIMER APELLIDO': 'Hess',
  'SEGUNDO APELLIDO': '',
  'full_name': 'HESS MICHAEL'}]

In [893]:
process.extractOne( 'lia',['n.'],scorer=fuzz.partial_ratio)

('n.', 100)

1    libardo antonio
Name: WOS_author, dtype: object

In [887]:
process.extractOne( unidecode.unidecode( l[i].get('NOMBRES').lower().split(' ')[0] )
                       ,  ll['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1],scorer=fuzz.partial_ratio )

('n.', 100, 0)

In [885]:
ll['WOS_author'].str.lower().apply(unidecode.unidecode).str.split(', ').str[-1]

0                 n.
1    libardo antonio
Name: WOS_author, dtype: object

In [953]:
i=13074
UDEA.loc[[i]].apply(missing_wos_authors_in_udea_authors,axis='columns')

13074    [{'NOMBRES': 'Libardo Antonio', 'SEGUNDO APELLIDO': 'Gaviria', 'UDEA_affiliation': ['Univ Antioquia, Colombia.'], 'WOS_author': ['Giraldo G, Libardo Antonio'], 'full_name': 'GIRALDO GAVIRIA LIBARD...
dtype: object

In [274]:
kk=UDEA.loc[[53]]

In [277]:
kk['UDEA_authors'].apply(len)

53    4
Name: UDEA_authors, dtype: int64

In [278]:
kk['authors_WOS'].apply(len)

53    2
Name: authors_WOS, dtype: int64

In [592]:
fuzz.token_sort_ratio(row['AU'][i], auwos[j] )

53

In [ ]:
fuzz.

In [549]:
fuzz.token_set_ratio?

Signature: fuzz.token_set_ratio(s1, s2, force_ascii=True, full_process=True)
Docstring: <no docstring>
File:      /usr/local/lib/python3.5/dist-packages/fuzzywuzzy/fuzz.py
Type:      function


In [ ]:
fuzz add_missing_authors_WOS(row):
    if len(row['AU'])>len(row['authors_WOS']):
        

In [433]:
UDEA['AU'].str.strip().str.split('\n').apply(len)==UDEA['authors_WOS'].apply(len)

0        False
1        False
2         True
3         True
4        False
5        False
6         True
7         True
8         True
9         True
10        True
11        True
12        True
13        True
14        True
15        True
16        True
17        True
18       False
19        True
20       False
21       False
22       False
23        True
24        True
25       False
26       False
27       False
28        True
29        True
         ...  
15614    False
15615    False
15616    False
15617     True
15618    False
15619    False
15620     True
15621    False
15622    False
15623    False
15624    False
15625     True
15626     True
15627    False
15628    False
15629     True
15630    False
15631    False
15632    False
15633    False
15634    False
15635     True
15636    False
15637    False
15638    False
15639    False
15640    False
15641    False
15642    False
15643    False
Length: 15644, dtype: bool

In [287]:
kk=pd.read_json('/home/restrepo/Downloads/WOS_SCI_SCP_PTJ_CTR.json.gz')

In [288]:
kk.loc[[53]][['authors_WOS','UDEA_authors']]

,authors_WOS,UDEA_authors
53,"[{'i': 0, 'WOS_author': 'Perez Patino, Ana Lucia', 'affiliation': ['Univ Antioquia, Dept Ingn Sistemas, Medellin, Colombia.']}]","[{'NOMBRES': 'Ana Lucia', 'CÉDULA': 43452493.0, 'WOS_author': ['Perez Patino, Ana Lucia', 'Perez, Ana Lucia', 'Perez, Ana L.'], 'full_name': 'PEREZ PATIÑO ANA LUCIA', 'INICIALES': 'A. L.', 'DEPART..."


In [298]:
kkk=kk[kk['TI']=='Effect of genetic line and sex on growth in guinea pigs (Cavia porcellus) [Efecto de la línea genética y el sexo sobre el crecimiento en cuyes (Cavia porcellus)]']

In [305]:
kkk[['AU','authors_WOS','UDEA_authors']]['authors_WOS'].apply(len)

194    2
Name: authors_WOS, dtype: int64

In [307]:
kkk[['AU','authors_WOS','UDEA_authors']]['UDEA_authors'].apply(len)

194    4
Name: UDEA_authors, dtype: int64

In [299]:
UDEA[UDEA['UDEA_authors'].apply(lambda l: [d for d in l if not d.get('WOS_author')]).apply(len)>0][['AU','authors_WOS','UDEA_authors']].loc[[53]]#.to_dict()

,AU,authors_WOS,UDEA_authors
53,"Posada, SL\nSolarte, CE\nNoguera, RR\n","[{'i': 0, 'WOS_author': 'Posada, S. L.', 'affiliation': ['Univ Antioquia, Grupo de Investigación en Ciencias Animales - GRICA, Medellín, Colombia']}, {'i': 1, 'WOS_author': 'Noguera, R. R.', 'affi...","[{'NOMBRES': 'Jaime Ricardo', 'CÉDULA': 98395898.0, 'WOS_author': ['Noguera, R. R.', 'Rosero Noguera, Ricardo', 'Noguera, Ricardo'], 'full_name': 'ROSERO NOGUERA JAIME RICARDO', 'INICIALES': 'J. R..."


Parte principal:
Adicione la información faltanta a `autores`

In [56]:
import itertools
def add_and_check_wos_authors(row,autores='autores',nombre='nombreAutor'):
    DEBUG=True
    wos_list =[ d.get('WOS_author')  for d in row.get('authors_WOS')]
    wos_aff  =[ d.get('affiliation') for d in row.get('authors_WOS')]

    udea_list=[ d.get('WOS_author') for d in row.get('UDEA_authors')]
    #Quality check: UdeA author in WOS_author
    if DEBUG: print(udea_list)
    for i in range(len(udea_list)):
    #for uli in udea_list:
        #print(uli)
        l=[]
        try:
            for ul in udea_list[i]:
                ur=process.extractOne(ul, wos_list,
                                     scorer=fuzz.ratio)
                if DEBUG: print(ul,ur)
                l.append(ur[1])
            urmax=max(l)
            #row.get('UDEA_authors').pop(i)
            #row.get('autores').pop(i)
            row.get('UDEA_authors')[i]['Q']=urmax
                #If failed remove
        except:
            print(row['DI'])
            pass
        if DEBUG: print('+++'*20)

    if DEBUG: print('='*80)
    try:
        udea_list = list(itertools.chain.from_iterable(udea_list))
    except TypeError:
        udea_list=[]

    #Complete missing authors
    for i in range( len(wos_list) ): 
        r=process.extractOne(wos_list[i], udea_list,
                                 scorer=fuzz.ratio)
            #if r and r[1]==100 → WOS_author is UDEA_author
        try:
            if r[1]<100:
                dd={}
                dd[nombre]=' '.join( wos_list[i].split(', ')[::-1] )
                dd['afiliacionWOS']=wos_aff[i][0]                
                    #Else → Add to authors column
                row[autores].append(dd)
                if DEBUG: print(wos_list[i],r)
        except:
            print('Failed',wos_list[i])
    pass

In [57]:
i=10003
udea.loc[[i]].apply(add_and_check_wos_authors, axis='columns')

[None, None]
10.1590/s0102-311x2008000400004
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
10.1590/s0102-311x2008000400004
++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Failed Grisales F, Lina María
Failed Giraldo G, Libardo Antonio


10003    None
dtype: object

In [165]:
udea[udea['DI']=='10.1590/s0102-311x2008000400004'][['authors_WOS',autores]]

,authors_WOS,autores
13074,"[{'i': 0, 'WOS_author': 'Grisales F, Lina María', 'affiliation': ['Univ Antioquia, Colombia.']}, {'i': 1, 'WOS_author': 'Giraldo G, Libardo Antonio', 'affiliation': ['Univ Antioquia, Colombia.']}]","[{'Identificación': 43746089, 'Tipo': 'Profesor vinculado', 'Nombres': 'Lina Maria Grisales Franco'}, {'Identificación': 43746089, 'Tipo': 'Profesor vinculado', 'Nombres': 'Lina Maria Grisales Fra..."


In [55]:
#udea.apply(add_and_check_wos_authors, axis='columns')

In [166]:
i=13074
udea[autores].loc[i]

[{'Identificación': 43746089,
  'Nombres': 'Lina Maria Grisales Franco',
  'Tipo': 'Profesor vinculado'},
 {'Identificación': 43746089,
  'Nombres': 'Lina Maria Grisales Franco',
  'Tipo': 'Profesor vinculado'}]

In [167]:
udea['authors_WOS'].loc[i]

[{'WOS_author': 'Grisales F, Lina María',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 0},
 {'WOS_author': 'Giraldo G, Libardo Antonio',
  'affiliation': ['Univ Antioquia, Colombia.'],
  'i': 1}]

In [168]:
udea['UDEA_authors'].loc[i]

[{'INICIALES': 'L. A.',
  'NOMBRE COMPLETO': 'Libardo Antonio Giraldo Gaviria',
  'NOMBRES': 'Libardo Antonio',
  'PRIMER APELLIDO': 'Giraldo',
  'SEGUNDO APELLIDO': 'Gaviria',
  'full_name': 'GIRALDO GAVIRIA LIBARDO ANTONIO'},
 {'CÉDULA': 43746089,
  'INICIALES': 'L. M.',
  'NOMBRE COMPLETO': 'Lina Maria Grisales Franco',
  'NOMBRES': 'Lina Maria',
  'PRIMER APELLIDO': 'Grisales',
  'SEGUNDO APELLIDO': 'Franco',
  'full_name': 'GRISALES FRANCO LINA MARIA'}]

In [64]:
UDEA[UDEA['UDEA_autores'].str.lower().str.contains('giraldo gaviria')][['UDEA_autores','UDEA_cedula']]

,UDEA_autores,UDEA_cedula
10003,GIRALDO GAVIRIA LIBARDO ANTONIO;GRISALES FRANCO LINA MARIA,0
1198,GIRALDO GAVIRIA LIBARDO ANTONIO;GRISALES FRANCO LINA MARIA,0


## TODO:
* Normalize cedula en UDEA → UDEA_authors desde AU y AU_PTJ
* Normalize authors_WOS en UDEA_authors
* Use Clasificación Colciencias de columnas `UDEA_`

In [148]:
UDEA.loc[5,'authors_WOS']

[{'WOS_author': 'Correa, Lorena E.',
  'affiliation': ['Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.'],
  'i': 0},
 {'WOS_author': 'Gomez, Maryory',
  'affiliation': ['Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.'],
  'i': 1},
 {'WOS_author': 'Echeverria, Felix',
  'affiliation': ['Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.'],
  'i': 2},
 {'WOS_author': 'Castano, Juan G.',
  'affiliation': ['Univ Antioquia, Grp Corros & Protecc CIDEMAT, Medellin, Colombia.'],
  'i': 3},
 {'WOS_author': 'Velilla, Esteban',
  'affiliation': ['Univ Antioquia, Grp Invest Manejo Eficiente Energia Elect, Medellin, Colombia.'],
  'i': 4}]

In [150]:
UDEA['UDEA_authors'].loc[0]

[{'CÉDULA': 18927154.0,
  'DEPARTAMENTO': 'Departamento de Ingeniería Eléctrica',
  'FACULTAD': 'Facultad de Ingeniería',
  'GRUPO': 'Grupo de Investigación en Materiales y Sistemas Energéticos TESLA',
  'INICIALES': 'E.',
  'NOMBRE COMPLETO': 'Edwin Garcia Quintero',
  'NOMBRES': 'Edwin',
  'PRIMER APELLIDO': 'Garcia',
  'SEGUNDO APELLIDO': 'Quintero',
  'WOS_affiliation': ['Univ Antioquia, Dept Ingn Elect, Medellin, Colombia.',
   'Univ Antioquia, Colombia.',
   'Department of Electrical Engineering, Univ Antioquia, Colombia'],
  'WOS_author': ['Garcia Quintero, Edwin',
   'Garcia-Quintero, E.',
   'Garcia-Quintero, Edwin',
   'Garcia, Edwin'],
  'full_name': 'GARCIA QUINTERO EDWIN'}]

In [259]:
UDEA['UDEA_authors'].shape

(12270,)

In [189]:
df=AU_PTJ.copy()
d={'full_name':'HOYOS FRANCO MARCO ANTONIO'}


,cedula,nombre
1587,70518213,HOYOS FRANCO MARCO ANTONIO


In [217]:
AU_PTJ[AU_PTJ['nombre'].str.lower().str.contains('hoyos franco marco antonio')
      ].reset_index(drop=True).loc[
                    0,'cedula']

70518213

UDEA.loc[[10003]].apply( add_authors_WOS_to_UDEA  )

In [96]:
pp[pp['nombre']=='HOYOS']

,cedula,nombre


In [106]:
AU[AU['NOMBRE COMPLETO'].str.lower()=='hoyos']

,CÉDULA,DEPARTAMENTO,FACULTAD,GRUPO,NOMBRE COMPLETO,NOMBRES,PRIMER APELLIDO,SEGUNDO APELLIDO


In [107]:
ppp.columns

Index(['año realiz', 'doi', 'fecha aplicación', 'idioma', 'item adic',
       'material', 'nombre', 'nombre revista o premio', 'nro autores', 'país',
       'procodigo', 'ptos', 'título', 'valor item'],
      dtype='object')

In [109]:
ppp[ppp['nombre']=='HOYOS']

,año realiz,doi,fecha aplicación,idioma,item adic,material,nombre,nombre revista o premio,nro autores,país,procodigo,ptos,título,valor item


In [157]:
au=AU[AU['CÉDULA']==int(618927154.0)].reset_index(drop=True)
au.empty

True

In [159]:
udea_au[list(new_format.columns)]

False

In [149]:
audea.loc[1].to_dict()

{'annio': 2008,
 'autores': [{'Identificación': 32508663,
   'Nombres': 'Del Consuelo Montes Ramirez',
   'Tipo': 'Profesor vinculado'}],
 'clasificacionColciencias': None,
 'doi': '',
 'idioma': 'Spanish',
 'issn': '0120-5609',
 'nombreRevista': 'INGENIERIA E INVESTIGACION',
 'numero': '3',
 'paginas': '41-46',
 'proyectoAsociado': None,
 'semestre': None,
 'tipoRevista': 'Revista Internacional Indexada',
 'tituloArticulo': 'Cement plant gaseous pollutant emission reduction technologies',
 'volumen': '28'}

In [69]:
exl = [{'id': 123, 'data': 'qwerty', 'indices': [1,10]}, {'id': 345, 'data': 'mnbvc', 'indices': [2,11]}]

In [70]:
with open("kk.json", "w") as outfile: 
    json.dump(exl, outfile)

In [75]:
ex.to_json('kk.json',orient='records')

In [76]:
cat kk.json

[{"annio":2019,"autores":[{"nombreAutor":"Jonathan Aguilar Bedoya","programaAcademico":["60178-MAESTR\u00cdA EN INGENIER\u00cdA MEC\u00c1NICA-INVESTIGACI\u00d3N"],"identificacionAutor":null,"grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Estudiante de posgrado"},{"nombreAutor":"Ainhoa Rubio Celemente","institucion":"Tecnol\u00f3gico de Antioquia. Instituci\u00f3n Universitaria","pais":"Colombia","grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Autor externo"},{"nombreAutor":"Laura Isable Velasquez Garcia","programaAcademico":["516-INGENIER\u00cdA MEC\u00c1NICA"],"grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Profesor ocasional"},{"nombreAutor":"Edwin Lenin Chica Arrieta","programaAcademico":["516-INGENIER\u00cdA MEC\u00c1NICA"],"identificacionAutor":"78761560","grupoInvestigacion":["Grupo de Energ\u00eda Alternativa"],"tipoAutor":"Profesor vinculado"}],"clasificacionColciencias":"A1","doi":"10.3390\/en12244679","id

In [77]:
pd.read_json('kk.json')

,annio,autores,clasificacionColciencias,doi,idioma,issn,nombreRevista,numero,paginas,proyectoAsociado,semestre,tipoRevista,tituloArticulo,volumen
0,2019,"[{'nombreAutor': 'Jonathan Aguilar Bedoya', 'programaAcademico': ['60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN'], 'tipoAutor': 'Estudiante de posgrado', 'identificacionAutor': None, 'grupo...",A1,10.3390/en12244679,Inglés,1996-1073,Energies,24,1-18,NaN,2019-2,Revista Internacional Indexada,Design and Optimization of a Multi-Element Hydrofoil for a Horizontal-Axis Hydrokinetic Turbine,12


In [78]:
with open(r"kk.json", "r") as read_file:
    data = json.load(read_file)

In [79]:
data

[{'annio': 2019,
  'autores': [{'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'identificacionAutor': None,
    'nombreAutor': 'Jonathan Aguilar Bedoya',
    'programaAcademico': ['60178-MAESTRÍA EN INGENIERÍA MECÁNICA-INVESTIGACIÓN'],
    'tipoAutor': 'Estudiante de posgrado'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'institucion': 'Tecnológico de Antioquia. Institución Universitaria',
    'nombreAutor': 'Ainhoa Rubio Celemente',
    'pais': 'Colombia',
    'tipoAutor': 'Autor externo'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'nombreAutor': 'Laura Isable Velasquez Garcia',
    'programaAcademico': ['516-INGENIERÍA MECÁNICA'],
    'tipoAutor': 'Profesor ocasional'},
   {'grupoInvestigacion': ['Grupo de Energía Alternativa'],
    'identificacionAutor': '78761560',
    'nombreAutor': 'Edwin Lenin Chica Arrieta',
    'programaAcademico': ['516-INGENIERÍA MECÁNICA'],
    'tipoAutor': 'Profesor vinculado'}],
  'clasificacionColci

In [80]:
with open("kkk.json", "w") as outfile: 
    json.dump(data, outfile) 

In [81]:
cat kkk.json

[{"doi": "10.3390/en12244679", "tipoRevista": "Revista Internacional Indexada", "numero": 24, "semestre": "2019-2", "annio": 2019, "idioma": "Ingl\u00e9s", "nombreRevista": "Energies", "issn": "1996-1073", "paginas": "1-18", "proyectoAsociado": null, "volumen": 12, "autores": [{"nombreAutor": "Jonathan Aguilar Bedoya", "programaAcademico": ["60178-MAESTR\u00cdA EN INGENIER\u00cdA MEC\u00c1NICA-INVESTIGACI\u00d3N"], "tipoAutor": "Estudiante de posgrado", "identificacionAutor": null, "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"]}, {"nombreAutor": "Ainhoa Rubio Celemente", "pais": "Colombia", "tipoAutor": "Autor externo", "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"], "institucion": "Tecnol\u00f3gico de Antioquia. Instituci\u00f3n Universitaria"}, {"nombreAutor": "Laura Isable Velasquez Garcia", "programaAcademico": ["516-INGENIER\u00cdA MEC\u00c1NICA"], "grupoInvestigacion": ["Grupo de Energ\u00eda Alternativa"], "tipoAutor": "Profesor ocasional"}, {"nombreAut